<a href="https://colab.research.google.com/github/marina554/excel_automation/blob/main/Log_Analyzer_Tool(%E3%83%AD%E3%82%B0%E6%8A%BD%E5%87%BA_%2B_%E9%9B%86%E8%A8%88_%2B_%E3%83%AC%E3%83%9D%E3%83%BC%E3%83%88%E7%94%9F%E6%88%90).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd
from datetime import datetime

LOG_DIR = "logs"
OUTPUT_DIR = "output"
OUTPUT_FILE = f"{OUTPUT_DIR}/log_report_{datetime.now().strftime('%Y%m%d')}.xlsx"

# ログフォーマット例:
# [2025-01-01 12:30:22] ERROR: Failed to connect to database
# [2025-01-01 12:31:05] WARNING: Slow response time detected

def parse_log_line(line):
    """ログ1行から情報を抽出"""
    try:
        timestamp = line.split("]")[0].replace("[", "")
        level = line.split("]")[1].split(":")[0].strip()
        message = line.split(":", 1)[1].strip()

        return timestamp, level, message
    except:
        return None, None, None


def read_logs(log_dir):
    """フォルダ内のログをすべて読み込む"""
    log_files = glob.glob(os.path.join(log_dir, "*.log"))
    records = []

    for file in log_files:
        with open(file, "r", encoding="utf-8") as f:
            for line in f:
                timestamp, level, message = parse_log_line(line)
                if timestamp and level:
                    records.append([timestamp, level, message])

    df = pd.DataFrame(records, columns=["timestamp", "level", "message"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    return df


def analyze_logs(df):
    """ログ分析（エラー別頻度、時間帯別分析など）"""
    # エラー種別カウント
    summary_by_level = df["level"].value_counts().reset_index()
    summary_by_level.columns = ["level", "count"]

    # 時間帯別集計（1時間ごと）
    df["hour"] = df["timestamp"].dt.hour
    summary_by_hour = df.groupby(["hour", "level"]).size().reset_index(name="count")

    return summary_by_level, summary_by_hour


def save_report(df, summary_by_level, summary_by_hour):
    """Excelレポートとして出力"""
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
        df.to_excel(writer, index=False, sheet_name="raw_logs")
        summary_by_level.to_excel(writer, index=False, sheet_name="by_level")
        summary_by_hour.to_excel(writer, index=False, sheet_name="by_hour")

    print(f"レポート出力完了: {OUTPUT_FILE}")


def main():
    print("=== ログ分析ツール ===")

    df = read_logs(LOG_DIR)

    summary_by_level, summary_by_hour = analyze_logs(df)

    save_report(df, summary_by_level, summary_by_hour)

    print("処理が完了しました！")


if __name__ == "__main__":
    main()
